# Model Composition with MLJFlux

This tutorial is available as a Jupyter notebook or julia script
[here](https://github.com/FluxML/MLJFlux.jl/tree/dev/docs/src/common_workflows/composition).

In this workflow example, we see how MLJFlux enables composing MLJ models with MLJFlux
models. We will assume a class imbalance setting and wrap an oversampler with a deep
learning model from MLJFlux.

In [1]:
using Pkg
Pkg.activate(@__DIR__);
Pkg.instantiate();

  Activating project at `~/GoogleDrive/Julia/MLJ/MLJFlux/docs/src/common_workflows/composition`


**Julia version** is assumed to be 1.10.*

### Basic Imports

In [2]:
using MLJ               # Has MLJFlux models
using Flux              # For more flexibility
import RDatasets        # Dataset source
import Random           # To create imbalance
import Imbalance        # To solve the imbalance
import Optimisers       # native Flux.jl optimisers no longer supported

### Loading and Splitting the Data

In [3]:
iris = RDatasets.dataset("datasets", "iris");
y, X = unpack(iris, ==(:Species), colname -> true, rng=123);
X = Float32.(X);      # To be compatible with type of network network parameters

To simulate an imbalanced dataset, we will take a random sample:

In [4]:
Random.seed!(803429)
subset_indices = rand(1:size(X, 1), 100)
X, y = X[subset_indices, :], y[subset_indices]
Imbalance.checkbalance(y)

versicolor: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 28 (65.1%) 
virginica:  ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 29 (67.4%) 
setosa:     ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 43 (100.0%) 


### Instantiating the model

Let's load `BorderlineSMOTE1` to oversample the data and `Standardizer` to standardize
it.

In [5]:
BorderlineSMOTE1 = @load BorderlineSMOTE1 pkg=Imbalance verbosity=0
NeuralNetworkClassifier = @load NeuralNetworkClassifier pkg=MLJFlux

[ Info: For silent loading, specify `verbosity=0`. 
import MLJFlux ✔


MLJFlux.NeuralNetworkClassifier

We didn't need to load Standardizer because it is a local model for MLJ (see
`localmodels()`)

In [6]:
clf = NeuralNetworkClassifier(
    builder=MLJFlux.MLP(; hidden=(5,4), σ=Flux.relu),
    optimiser=Optimisers.Adam(0.01),
    batch_size=8,
    epochs=50,
    rng=42,
)

NeuralNetworkClassifier(
  builder = MLP(
        hidden = (5, 4), 
        σ = NNlib.relu), 
  finaliser = NNlib.softmax, 
  optimiser = Adam(0.01, (0.9, 0.999), 1.0e-8), 
  loss = Flux.Losses.crossentropy, 
  epochs = 50, 
  batch_size = 8, 
  lambda = 0.0, 
  alpha = 0.0, 
  rng = 42, 
  optimiser_changes_trigger_retraining = false, 
  acceleration = ComputationalResources.CPU1{Nothing}(nothing))

First we wrap the oversampler with the neural network via the `BalancedModel`
construct. This comes from `MLJBalancing` And allows combining resampling methods with
MLJ models in a sequential pipeline.

In [7]:
oversampler = BorderlineSMOTE1(k=5, ratios=1.0, rng=42)
balanced_model = BalancedModel(model=clf, balancer1=oversampler)
standarizer = Standardizer()

Standardizer(
  features = Symbol[], 
  ignore = false, 
  ordered_factor = false, 
  count = false)

Now let's compose the balanced model with a standardizer.

In [8]:
pipeline = standarizer |> balanced_model

ProbabilisticPipeline(
  standardizer = Standardizer(
        features = Symbol[], 
        ignore = false, 
        ordered_factor = false, 
        count = false), 
  balanced_model_probabilistic = BalancedModelProbabilistic(
        model = NeuralNetworkClassifier(builder = MLP(hidden = (5, 4), …), …), 
        balancer1 = BorderlineSMOTE1(m = 5, …)), 
  cache = true)

By this, any training data will be standardized then oversampled then passed to the
model. Meanwhile, for inference, the standardizer will automatically use the training
set's mean and std and the oversampler will be transparent.

### Training the Composed Model

It's indistinguishable from training a single model.

In [9]:
mach = machine(pipeline, X, y)
fit!(mach)
cv=CV(nfolds=5)
evaluate!(mach, resampling=cv, measure=accuracy)

[ Info: Training machine(ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:balanced_model_probabilistic, …).
[ Info: Training machine(BorderlineSMOTE1(m = 5, …), …).
[ Info: Training machine(:model, …).
[ Info: After filtering, the mapping from each class to number of borderline points is ("virginica" => 1, "versicolor" => 2).
Progress: 100%|█████████████████████████████████████████████████████| Time: 0:00:00
Progress:  67%|███████████████████████████████████▍                 |  ETA: 0:00:01
[ Info: After filtering, the mapping from each class to number of borderline points is ("virginica" => 1, "versicolor" => 2).
Optimising neural net: 100%[=========================] Time: 0:00:12
[ Info: After filtering, the mapping from each class to number of borderline points is ("virginica" => 3, "versicolor" => 1).
[ Info: After filtering, the mapping from each class to number of borderline po

PerformanceEvaluation object with these fields:
  model, measure, operation,
  measurement, per_fold, per_observation,
  fitted_params_per_fold, report_per_fold,
  train_test_rows, resampling, repeats
Extract:
┌────────────┬──────────────┬─────────────┐
│ measure    │ operation    │ measurement │
├────────────┼──────────────┼─────────────┤
│ Accuracy() │ predict_mode │ 0.99        │
└────────────┴──────────────┴─────────────┘
┌────────────────────────────┬─────────┐
│ per_fold                   │ 1.96*SE │
├────────────────────────────┼─────────┤
│ [1.0, 1.0, 0.95, 1.0, 1.0] │ 0.0219  │
└────────────────────────────┴─────────┘


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*